In [39]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Word lists and lexicons in nltk: https://www.nltk.org/howto/corpus.html#word-lists-and-lexicons
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("This is a really great tweet!")

# Output:
# [nltk_data] Downloading package vader_lexicon to /root/nltk_data...
# [nltk_data]   Package vader_lexicon is already up-to-date!
# {'compound': 0.6893, 'neg': 0.0, 'neu': 0.461, 'pos': 0.539}

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ZyroY\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 0.461, 'pos': 0.539, 'compound': 0.6893}

In [40]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load the dataset
df = pd.read_csv('tw_tweets_users_media_places.csv')
df.head()




,tweet_id,text,context_annotations_count,count_annotations,count_cashtags,count_hashtags,count_mentions,count_urls,created_at_tweet,lang,...,location,protected,verified,media_type,height,width,preview_image_url,country,name_place,place_type
0,1440484799970304000,This was my grandson this morning (w/autism)! ...,1,0.0,0.0,0.0,0.0,1.0,2021-09-22T01:15:13.000Z,en,...,"Victoria, BC",False,False,photo,405,813,https://pbs.twimg.com/media/E_2hSs4UcAAIOK5.jpg,Canada,Langford,city
1,1439618825171963904,Wow!! Been into #York for the first time since...,2,2.0,0.0,3.0,0.0,1.0,2021-09-19T15:54:09.000Z,en,...,"Hessay, York",False,False,photo,2048,1536,https://pbs.twimg.com/media/E_qNsE1X0AQmoK_.jpg,United Kingdom,Hessay,city
2,1248872872837332992,Sad number of ppl who lost life due to covid-1...,3,0.0,0.0,0.0,0.0,1.0,2020-04-11T07:17:50.000Z,en,...,"Maidstone, South East",False,False,photo,288,278,https://pbs.twimg.com/media/EVTjQcoXsAAlrfq.jpg,United Kingdom,Maidstone,city
3,1250729294051053568,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-16T10:14:35.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVt7pYTXkAMGzxj.jpg,United Kingdom,Maidstone,city
4,1249612131433095168,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-13T08:15:23.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVeDlp7X0AMuN6X.jpg,United Kingdom,Maidstone,city


In [41]:
# First, check the data types
print("\nData types in the dataset:")
print(df.dtypes)
# Select only numerical columns


Data types in the dataset:
tweet_id                       int64
text                          object
context_annotations_count      int64
count_annotations            float64
count_cashtags               float64
count_hashtags               float64
count_mentions               float64
count_urls                   float64
created_at_tweet              object
lang                          object
likes                          int64
quotes                         int64
referenced_tweet_count         int64
replies                        int64
reply_settings                object
retweets                       int64
source                        object
terms                         object
username                      object
created_at_author             object
followers_count                int64
following_count                int64
tweet_count                    int64
listed_count                   int64
location                      object
protected                       bool
verified  

In [42]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Ensure 'text' column is string and handle NaNs
df['text'] = df['text'].astype(str).fillna('')

# Initialize sentiment columns
df['sentiment_overall'] = 0.0
df['sentiment_neg'] = 0.0
df['sentiment_neu'] = 0.0
df['sentiment_pos'] = 0.0

# Perform sentiment analysis
for row in df.itertuples():
    sentiment = sia.polarity_scores(row.text)
    df.loc[row.Index, 'sentiment_overall'] = sentiment['compound']
    df.loc[row.Index, 'sentiment_neg'] = sentiment['neg']
    df.loc[row.Index, 'sentiment_neu'] = sentiment['neu']
    df.loc[row.Index, 'sentiment_pos'] = sentiment['pos']


In [43]:
df = df.select_dtypes(include=['number', 'float64', 'int64'])

In [ ]:
import statsmodels.api as sm


y = df['retweets']
X = df.drop(columns=['retweets', 'sentiment_overall', 'context_annotations_count', 'count_annotations', 'count_cashtags', 'count_hashtags', 
                     'count_mentions', 'count_urls', 'referenced_tweet_count', 'listed_count', 'width', 'height', 'tweet_count', 
                     'sentiment_neu', 'sentiment_neg', 'sentiment_pos', 'likes', 'quotes', 'replies', 'following_count']).assign(const=1)

print(sm.OLS(y, X).fit().summary())

# Output:
# R-squared: 0.246

                            OLS Regression Results                            
Dep. Variable:               retweets   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     32.46
Date:                Tue, 01 Apr 2025   Prob (F-statistic):           2.02e-08
Time:                        18:51:04   Log-Likelihood:                -2333.5
No. Observations:                 534   AIC:                             4671.
Df Residuals:                     532   BIC:                             4680.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
tweet_id         3.061e-18   6.41e-19     

#### retweets=(−1.601 * 10^-12) + ((3.061 * 10^-18) * tweet_id) + ((8.242 * 10^-5) * followers_count)
#### Our model goes as above

#### The significant predictors of retweets are **followers_count** (p < 0.001) and **tweet_id** (p < 0.001). More followers correlate with more retweets, while tweet_id likely reflects time-based effects. Sentiment variables were not significant, as they were not included in the final model.

In [46]:
# Create a blank DataFrame to store the results
df_vif = pd.DataFrame(columns=['VIF', 'Tolerance'])
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns, drop_first=True)

df = df.astype(float)

for col in df.drop(columns=['retweets']):
  y = df[col]
  X = df.drop(columns=[col, 'retweets']).assign(const=1)
  
  r_squared = sm.OLS(y, X).fit().rsquared

  if r_squared < 1:  # Prevent division by zero runtime error
      df_vif.loc[col] = [1/(1 - r_squared), 1 - r_squared]
  else:
      df_vif.loc[col] = [np.inf, 1 - r_squared]  # Use np.inf instead of string 'infinity'

# Convert VIF column to numeric to ensure proper sorting
df_vif['VIF'] = pd.to_numeric(df_vif['VIF'])

# Now sort the values
df_vif.sort_values(by=['VIF'], ascending=False)

c:\Users\ZyroY\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


,VIF,Tolerance
count_mentions,inf,NaN
listed_count,9.683661,0.103267
tweet_id,1.104726,0.905201
following_count,1.079818,0.926082
followers_count,1.071346,0.933405
quotes,1.070223,0.934385
likes,1.028131,0.972638
sentiment_neg,1.020121,0.980276
sentiment_overall,1.016764,0.983512
width,1.016566,0.983704
